In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key= os.getenv('GOOGLE_API_KEY')
if not api_key :
    raise (' Not key')

In [2]:
import os
import glob
from langchain_core.documents import Document

# Đường dẫn thư mục Dataset
DATASET_DIR = './Dataset_economy'

# Đọc tất cả file .txt từ Dataset_economy (bao gồm các thư mục con)
documents = []
for filepath in glob.glob(os.path.join(DATASET_DIR, '**', '*.txt'), recursive=True):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        if content:  # Bỏ qua file rỗng
            # Lấy tên danh mục từ đường dẫn thư mục
            parts = filepath.replace(DATASET_DIR, '').strip('/').split('/')
            category = parts[0] if len(parts) > 1 else 'unknown'
            
            documents.append(Document(
                page_content=content,
                metadata={
                    "source": filepath,
                    "category": category,
                    "filename": os.path.basename(filepath)
                }
            ))
    except Exception as e:
        print(f"Lỗi đọc file {filepath}: {e}")

print(f"Đã load {len(documents)} văn bản từ {DATASET_DIR}")

Đã load 27682 văn bản từ ./Dataset_economy


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
model_kwargs = {'device': 'cuda'}

encode_kwargs = {'normalize_embeddings': True}
model='keepitreal/vietnamese-sbert'
embeddings=HuggingFaceEmbeddings(model_name=model,
                                 model_kwargs=model_kwargs,
                                 encode_kwargs=encode_kwargs)

2026-02-25 20:49:04.737367: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-25 20:49:04.774707: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-25 20:49:05.541076: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma.vectorstores import Chroma

doc=documents

split=RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 300,
    separators=['\n\n','\n','.',',',' ']
)
doc_split= split.split_documents(doc)

if os.path.exists('./chroma_economy_db'):
    chroma=Chroma(
        embedding_function=embeddings,
        persist_directory='./chroma_economy_db'
    )
else:
    chroma=Chroma.from_documents(
        doc_split,
        embedding=embeddings,
        persist_directory='./chroma_economy_db'
    )
retriever_cosine=chroma.as_retriever(search_kwargs={'k':30})

In [5]:
import numpy as np
from scipy.spatial.distance import cdist

def energy_base_distance(X,Y):
    
    X=np.asarray(X)
    Y=np.asarray(Y)
#Tuong tac cheo
    d_xy=cdist(X,Y,metric='euclidean')
    E_xy=np.mean(d_xy)
#Tu nang x
    d_xx=cdist(X,X,metric='euclidean')
    E_xx=np.mean(d_xx)
#Tu nang y
    d_yy=cdist(Y,Y,metric='euclidean')
    E_yy=np.mean(d_yy)
#Energy_distance
    ED=2*E_xy - E_xx - E_yy

    return max(0,ED)

In [6]:
X = [[1,1],[2,3]]
Y = [[1,1],[2,3]]

ed=energy_base_distance(X,Y)
print(f'{ed}')

0


In [7]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_with_energy_clustering(query, retriever, embeddings, k_clusters=7):

    docs=retriever.invoke(query)

    context=[doc.page_content for doc in docs]

    doc_vector= np.array(embeddings.embed_documents(context))
    query_vector=np.array(embeddings.embed_query(query)).reshape(1,-1)
#kiem loi
    sims = cosine_similarity(query_vector, doc_vector)[0]
    print("Max cosine:", np.max(sims))
    if np.max(sims) < 0.40:
        return [] 
#tao cum
    actual_k=min(k_clusters,len(doc_vector))
    kmeans=KMeans(n_clusters= actual_k, random_state=42, n_init='auto')
    labels=kmeans.fit_predict(doc_vector)
#kiem tra e
    best_energy=float('inf')
    best_cluster_idx=-1
    for i in range(actual_k):
        indices=np.where(labels==i)[0]
        cluster_vector=doc_vector[indices]

        energy=energy_base_distance(query_vector,cluster_vector)

        if energy < best_energy:
            best_energy=energy
            best_cluster_idx=i
#tim idx e thap
    win_index=np.where(labels==best_cluster_idx)[0]
    print(f'{best_energy}')
#doan context e thap
    final_docs=[context[i] for i in win_index]

    return final_docs

In [8]:
query='Mục tiêu doanh thu và lợi nhuận năm 2024 của Phục Hưng Holdings (PHC) được đặt ra là bao nhiêu?'
context_doc=retrieve_with_energy_clustering(query,retriever_cosine,embeddings)
print(f'{context_doc}')

Max cosine: 0.6233538046911788
1.2872645809823173
['Dựa trên công bố thông tin hiện hành, ghi nhận kế hoạch một số doanh nghiệp có kế hoạch phát hành trái phiếu trong thời gian tới bao gồm chủ yếu là các ngân hàng như HB Bank, VietBank và hai doanh nghiệp trong ngành bất động sản bao gồm Vingroup và Tổng công ty Đầu tư và Phát triển (DIG). Tổng giá trị phát hành hơn 10.000 tỷ đồng.\nNhìn dài hơn cả năm 2024, mặc dù rất khó để có thể dự báo được giá trị phát hành dự kiến cả năm 2024 nhưng FiinRatings vẫn cho rằng triển vọng cho kênh huy động trái phiếu doanh nghiệp năm 2024 sẽ sôi động hơn năm 2023 vì những lý do dưới đây.', 'Với triển vọng như vậy, các kịch bản kinh tế mà Tổng cục Thống kê đưa ra là như thế nào, thưa bà?\nTại kỳ họp thứ 6, Quốc hội khóa XV, Quốc hội đã thông qua Nghị quyết số 103/2023/QH15 về Kế hoạch phát triển kinh tế - xã hội năm 2024 ban hành ngày 9/11/2023, với mục tiêu tốc độ tăng trưởng tổng sản phẩm trong nước (GDP) đạt từ 6 - 6,5%.\nTrên cơ sở số liệu năm 2023

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
def answer(question,context,embeddings):
    # model='gemini-2.0-flash'
    model='gemini-2.5-flash'
    llm=ChatGoogleGenerativeAI(model=model)
    context_ed= retrieve_with_energy_clustering(question,context,embeddings)

    if not context_ed:
        return 'Khong co thong tin'

    template='''Dựa vào các đoạn văn sau:

 {context}

 Trả lời câu hỏi: {query}

 YÊU CẦU BẮT BUỘC:
 - Trả lời NGẮN GỌN, đi thẳng vào vấn đề
 - KHÔNG lặp lại câu hỏi trong câu trả lời
 - KHÔNG bắt đầu bằng "Dựa vào đoạn văn..."
 - Trích dẫn ĐẦY ĐỦ tất cả số liệu liên quan (%, tỷ đồng, mức tăng/giảm, mức đích...)
 - Nếu có nhiều thông tin liên quan, tổng hợp HẾT
 - Nếu không có thông tin, trả lời: "Không có thông tin về..."

 Câu trả lời:'''
    prompt=ChatPromptTemplate.from_template(template)
    chain = prompt | llm | StrOutputParser()

    return chain.invoke({'context' : context_ed,
                        'query':question                    
    })

In [10]:
y='So sánh tốc độ tăng trưởng giữa doanh thu và lợi nhuận của Hòa Phát năm 2025?'
tl=answer(y,retriever_cosine,embeddings)
tl

Max cosine: 0.6271532973161205
1.2122068593829765


'Không có thông tin về Hòa Phát năm 2025.'

In [11]:
# while True:
#     q=input('nhap cau hoi')
#     if q.strip().lower() == "quit":        
#         break
#     else:
#         tl=answer(q,retriever_cosine,embeddings)
#         print(f'cau tra loi: {tl}')